In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np 

Mounted at /content/drive


In [ ]:
TempData="/content/drive/MyDrive/P2M/Data/temperaturemod.csv"
MvtData="/content/drive/MyDrive/P2M/Data/mvtmod.csv"
ScrapData="/content/drive/MyDrive/P2M/Data/ScrapedData.csv"

# Data Reading


In [ ]:
mvt = pd.read_csv(MvtData)
temp = pd.read_csv(TempData)
ScrapTemp = pd.read_csv(ScrapData)

# Merge MVT + Tmp

Merge MVT and Temperature 

In [ ]:
output = pd.merge(mvt, temp, 
                   on='Time_stamp', 
                   how='outer')
output.describe()

,Id_box_x,Emp_box_x,Indicateur_x,Mvt,Id_box_y,Emp_box_y,Indicateur_y,Temp
count,4346008.0,4346008.0,4346008.0,4346008.0,4349584.0,4349584.0,4349584.0,4.349584e+06
mean,543.0,168.0,207.0,1.0,542.0,168.0,67.0,2.081546e+01
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.350488e+00
min,543.0,168.0,207.0,1.0,542.0,168.0,67.0,-1.270000e+02
25%,543.0,168.0,207.0,1.0,542.0,168.0,67.0,1.700000e+01
50%,543.0,168.0,207.0,1.0,542.0,168.0,67.0,2.000000e+01
75%,543.0,168.0,207.0,1.0,542.0,168.0,67.0,2.400000e+01
max,543.0,168.0,207.0,1.0,542.0,168.0,67.0,3.100000e+01


In [ ]:
output

,Id_box_x,Emp_box_x,Indicateur_x,Mvt,Time_stamp,Id_box_y,Emp_box_y,Indicateur_y,Temp
0,543.0,168.0,207.0,1.0,2021-08-10 00:00:08,NaN,NaN,NaN,NaN
1,543.0,168.0,207.0,1.0,2021-08-10 00:00:14,NaN,NaN,NaN,NaN
2,543.0,168.0,207.0,1.0,2021-08-10 00:00:25,NaN,NaN,NaN,NaN
3,543.0,168.0,207.0,1.0,2021-08-10 00:00:40,NaN,NaN,NaN,NaN
4,543.0,168.0,207.0,1.0,2021-08-10 00:00:40,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6595848,NaN,NaN,NaN,NaN,2022-01-31 08:59:29,542.0,168.0,67.0,16.0
6595849,NaN,NaN,NaN,NaN,2022-01-31 08:59:36,542.0,168.0,67.0,16.0
6595850,NaN,NaN,NaN,NaN,2022-01-31 08:59:43,542.0,168.0,67.0,16.0
6595851,NaN,NaN,NaN,NaN,2022-01-31 08:59:53,542.0,168.0,67.0,16.0


# Data Cleaning after merge

In [ ]:
output.head()

,Id_box_x,Emp_box_x,Indicateur_x,Mvt,Time_stamp,Id_box_y,Emp_box_y,Indicateur_y,Temp
0,543.0,168.0,207.0,1.0,2021-08-10 00:00:08,NaN,NaN,NaN,NaN
1,543.0,168.0,207.0,1.0,2021-08-10 00:00:14,NaN,NaN,NaN,NaN
2,543.0,168.0,207.0,1.0,2021-08-10 00:00:25,NaN,NaN,NaN,NaN
3,543.0,168.0,207.0,1.0,2021-08-10 00:00:40,NaN,NaN,NaN,NaN
4,543.0,168.0,207.0,1.0,2021-08-10 00:00:40,NaN,NaN,NaN,NaN


In [ ]:
output.columns

Index(['Id_box_x', 'Emp_box_x', 'Indicateur_x', 'Mvt', 'Time_stamp',
       'Id_box_y', 'Emp_box_y', 'Indicateur_y', 'Temp'],
      dtype='object')

In [ ]:
output.sort_values(by=['Time_stamp'], inplace=True)

In [ ]:
output.set_index('Time_stamp',inplace=True)

In [ ]:
#Fill data with zeros for when no movement detected
output['Mvt'] = output['Mvt'].fillna(0)

In [ ]:
output.bfill(axis ='rows',inplace= True)

In [ ]:
output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6595853 entries, 4346008 to 4346007
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Id_box_x      float64
 1   Emp_box_x     float64
 2   Indicateur_x  float64
 3   Mvt           float64
 4   Time_stamp    object 
 5   Id_box_y      float64
 6   Emp_box_y     float64
 7   Indicateur_y  float64
 8   Temp          float64
dtypes: float64(8), object(1)
memory usage: 503.2+ MB


In [ ]:
#Check Null values
output.isnull().sum().sort_values(ascending = False)

Id_box_y        1
Emp_box_y       1
Indicateur_y    1
Temp            1
Id_box_x        0
Emp_box_x       0
Indicateur_x    0
Mvt             0
Time_stamp      0
dtype: int64

In [ ]:
#Drop uneeded columns
output = output.drop(columns="Id_box_x")
output = output.drop(columns="Id_box_y")
output = output.drop(columns="Emp_box_x")
output = output.drop(columns="Emp_box_y")
output = output.drop(columns="Indicateur_x")
output = output.drop(columns="Indicateur_y")

# Merge Scrapped Data 

Merge Exterior Temperature

In [ ]:
ScrapTemp.describe()

,TempExt
count,1.229081e+07
mean,2.047393e+01
std,7.545223e+00
min,0.000000e+00
25%,1.500000e+01
50%,2.000000e+01
75%,2.600000e+01
max,4.800000e+01


In [ ]:
def check_space(string):
    count = 0
    for i in range(0, len(string)):
        if string[i] == " ":
            count += 1
         
    return count
#check the number of spaces in column time_stamp    
string = output.Time_stamp[1]
print(string)
print("number of spaces ",check_space(string))

2021-08-10 00:00:14
number of spaces  1


In [ ]:
output[['Date','Time']]=output.Time_stamp.str.split(" ",expand=True)
output = output.drop(columns="Time_stamp")
output.columns

Index(['Mvt', 'Temp', 'Date', 'Time'], dtype='object')

In [ ]:
FinalData = pd.merge(output, ScrapTemp, 
                   on=['Date', 'Time'], 
                   how='inner')
# displaying result
FinalData.describe()

,Mvt,Temp,TempExt
count,3.729875e+06,3.729875e+06,3.729875e+06
mean,6.528305e-01,2.200468e+01,1.932472e+01
std,4.760703e-01,4.323551e+00,7.289226e+00
min,0.000000e+00,-1.270000e+02,0.000000e+00
25%,0.000000e+00,1.800000e+01,1.400000e+01
50%,1.000000e+00,2.100000e+01,1.900000e+01
75%,1.000000e+00,2.700000e+01,2.500000e+01
max,1.000000e+00,3.100000e+01,4.800000e+01


# Merged Data Cleaning

In [ ]:
FinalData.head()

,Mvt,Temp,Date,Time,TempExt
0,0.0,29.0,2021-08-10,00:00:04,36
1,1.0,30.0,2021-08-10,00:00:08,36
2,0.0,30.0,2021-08-10,00:00:12,36
3,1.0,29.0,2021-08-10,00:00:14,36
4,0.0,29.0,2021-08-10,00:00:21,36


In [ ]:
df=FinalData

In [ ]:
df=df.sort_values(by=['Date','Time'])

In [ ]:
df

,Mvt,Temp,Date,Time,TempExt
0,0.0,29.0,2021-08-10,00:00:04,36
1,1.0,30.0,2021-08-10,00:00:08,36
2,0.0,30.0,2021-08-10,00:00:12,36
3,1.0,29.0,2021-08-10,00:00:14,36
4,0.0,29.0,2021-08-10,00:00:21,36
...,...,...,...,...,...
3729870,0.0,17.0,2021-12-31,23:59:53,10
3729871,1.0,17.0,2021-12-31,23:59:54,10
3729872,1.0,17.0,2021-12-31,23:59:56,10
3729873,0.0,17.0,2021-12-31,23:59:58,10


In [ ]:
df.duplicated().any()

True

In [ ]:
sthg=df[df.duplicated(subset=['Time','Date'], keep=False)]
sthg

In [ ]:
df=df.drop_duplicates()

In [ ]:
df.duplicated().any()

False

In [ ]:
df.isnull().sum()

Mvt        0
Temp       0
Date       0
Time       0
TempExt    0
dtype: int64

In [ ]:
df.describe()

,Mvt,Temp,TempExt
count,3.722750e+06,3.722750e+06,3.722750e+06
mean,6.526393e-01,2.200485e+01,1.932441e+01
std,4.761316e-01,4.324419e+00,7.290519e+00
min,0.000000e+00,-1.270000e+02,0.000000e+00
25%,0.000000e+00,1.800000e+01,1.400000e+01
50%,1.000000e+00,2.100000e+01,1.900000e+01
75%,1.000000e+00,2.700000e+01,2.500000e+01
max,1.000000e+00,3.100000e+01,4.800000e+01


# Final Data saving

In [ ]:
df.columns

Index(['Mvt', 'Temp', 'Date', 'Time', 'TempExt'], dtype='object')

In [ ]:
Data="/content/drive/MyDrive/P2M/Data/Data.csv"
headerlist=['Mvt', 'Temp', 'Date', 'Time', 'TempExt']
df.to_csv(Data, header=headerlist, index=False)

In [ ]:
df.columns

Index(['Mvt', 'Temp', 'Date', 'Time', 'TempExt'], dtype='object')

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/P2M/Data/Data.csv")
data.describe()

,Mvt,Temp,TempExt
count,3.722750e+06,3.722750e+06,3.722750e+06
mean,6.526393e-01,2.200485e+01,1.932441e+01
std,4.761316e-01,4.324419e+00,7.290519e+00
min,0.000000e+00,-1.270000e+02,0.000000e+00
25%,0.000000e+00,1.800000e+01,1.400000e+01
50%,1.000000e+00,2.100000e+01,1.900000e+01
75%,1.000000e+00,2.700000e+01,2.500000e+01
max,1.000000e+00,3.100000e+01,4.800000e+01
